NN on dcdc non bdd 

Fully connected


In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

import matplotlib.pyplot as plt
# plot param
%matplotlib inline
plt.rcParams['figure.figsize'] = (22.0, 12.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

First, manipulate the input and output data

Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [2]:
# parse the controller file
f = open('controller.scs', "r")

In [3]:
lines = []

Skip the header part and go straight to the state-action pair

In [4]:
for line in f:
    if '#MATRIX:DATA\n' in line:                
        for line in f: # now you are at the lines you want
            # skip the #BEGIN 
            # read the state-actions
            lines = f.readlines()

In [5]:
# take the state as the train dataset
ltrain_dataset = []
for x in lines:
    ltrain_dataset.append(x.split(' ')[0])
del ltrain_dataset[-1] # delete the string #END at the end of the file 

In [6]:
# convert to the numpy array with float32 data type
train_dataset = np.asarray(ltrain_dataset)
train_dataset = train_dataset.astype(np.float32)
train_dataset.shape

(593089,)

In [7]:
# take the action(s) [column 1:-1] / the rest of the integer except the state
ltrain_label = []
for x in lines:
    ltrain_label.append(x.split(' ')[1:-1])
del ltrain_label[-1] # delete the string #END at the end of the file  

In [8]:
# convert to numpy array, note that the result is still not in one hot encoding format
train_label = np.asarray(ltrain_label)
train_label

array([list(['1']), list(['1']), list(['1']), ..., list(['0']),
       list(['0']), list(['0'])], dtype=object)

Iterate from here

In [9]:
# define number of samples
num_samples = train_dataset.shape[0]
num_samples

593089

In [10]:
# create now array to be filled by the encoded label
train_label_hot = np.zeros([num_samples,2], dtype=np.float32)
train_label_hot.shape

(593089, 2)

In [11]:
ltrain_label[0]

['1']

In [12]:
# encode label to one hot encoding format
for i in range(num_samples):
    if train_label[i] == ['0']:
        train_label_hot[i] = [1, 0]
    elif train_label[i] == ['1']:
        train_label_hot[i] = [0, 1]
    elif train_label[i] == ['0','1']:
        train_label_hot[i] = [1, 1]

Small dataset

In [21]:
# define the new number of samples for debugging purpose
num_samples = 40
# define batch offset to limit the desired dataset index
batch_off = 20000+16800-87-25
# slice the train label to the desired index
train_label_hot = train_label_hot[batch_off:batch_off + num_samples]
train_label_hot.shape

(40, 2)

In [22]:
# do the same thing to the dataset and add dimension on the column
train_dataset_reform = train_dataset[batch_off:batch_off + num_samples, None]
train_dataset_reform.shape

(40, 1)

Execute this to get the full batch

In [13]:
train_dataset_reform = train_dataset[:, None]
train_dataset_reform.shape

(593089, 1)

# NN

In [14]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])


In [67]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(predictions == labels) / predictions.shape[1] / predictions.shape[0])

In [15]:
train_dataset_reform[0]

array([459.], dtype=float32)

In [16]:
train_dataset_reform[-1]

array([638857.], dtype=float32)

In [17]:
train_label_hot[0]

array([0., 1.], dtype=float32)

In [18]:
train_label_hot[-1]

array([1., 0.], dtype=float32)

In [19]:
num_samples

593089

In [20]:
num_label = 2

In [70]:
batch_size = int(num_samples/100)
hidden_size = 1024
hidden_size2 = 512
hidden_size3 = 256
hidden_size4 = 128

# 20, 100, 500, 1000, 2000

# graph = tf.Graph()
# with tf.device('/device:GPU:0'):
# with graph.as_default():
# Input data. For the training data, we use a placeholder that will be fed
# at run time with a training minibatch.
tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, 1))
tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_label))
# tf_valid_dataset = tf.constant(valid_dataset)
tf_test_dataset = tf.placeholder(tf.float32, shape=(batch_size, num_label))
tf_test_labels = tf.placeholder(tf.float32, shape=(batch_size, num_label))

# Variables.
weights1 = tf.Variable(tf.truncated_normal([1, hidden_size]))
biases1 = tf.Variable(tf.zeros([hidden_size]))

# weights2 = tf.Variable(tf.truncated_normal([hidden_size, num_label]))
# biases2 = tf.Variable(tf.zeros([num_label]))

weights2 = tf.Variable(tf.truncated_normal([hidden_size, hidden_size2]))
biases2 = tf.Variable(tf.zeros([hidden_size2]))

weights3 = tf.Variable(tf.truncated_normal([hidden_size2, hidden_size3]))
biases3 = tf.Variable(tf.zeros([hidden_size3]))

# weights4 = tf.Variable(tf.truncated_normal([hidden_size3, num_label]))
# biases4 = tf.Variable(tf.zeros([num_label]))
weights4 = tf.Variable(tf.truncated_normal([hidden_size3, hidden_size4]))
biases4 = tf.Variable(tf.zeros([hidden_size4]))

weights5 = tf.Variable(tf.truncated_normal([hidden_size4, num_label]))
biases5 = tf.Variable(tf.zeros([num_label]))

# Training computation.
logits1 = tf.matmul(tf_train_dataset, weights1) + biases1  
relu_act_func = tf.nn.relu(logits1)
logits2 = tf.matmul(relu_act_func, weights2) + biases2

relu_act_func2 = tf.nn.relu(logits2)
logits3 = tf.matmul(relu_act_func2, weights3) + biases3

relu_act_func3 = tf.nn.relu(logits3)
logits4 = tf.matmul(relu_act_func3, weights4) + biases4

relu_act_func4 = tf.nn.relu(logits4)
logits5 = tf.matmul(relu_act_func4, weights5) + biases5

logits = logits5

# loss calculation
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits5))
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf_train_labels, logits=logits5))

# Predictions for the training, validation, and test data.
# train_prediction = tf.nn.softmax(logits5)
train_prediction = tf.round(tf.sigmoid(logits5))

# loss = tf.reduce_mean(tf.losses.absolute_difference(labels=tf_train_labels, predictions=train_prediction))

loss_summary = tf.summary.scalar('loss', loss)

# Optimizer.
# optimizer = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)
optimizer = tf.train.AdamOptimizer(2).minimize(loss)

# Predictions for the training, validation, and test data.
# train_prediction = tf.nn.softmax(logits4)

#valid_prediction = tf.nn.softmax(
#    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
# test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(
  #  tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3), weights4)+biases4)
 
# test_prediction = tf.sigmoid(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(
#        tf_test_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3), weights4)+biases4),weights5) + biases5)
     
    # writer = tf.summary.FileWriter('./vehicle', tf.get_default_graph())


In [71]:
batch_size

5930

In [72]:
num_samples

593089

In [73]:
iteration_one_epoch = np.floor(num_samples / batch_size)
iteration_one_epoch

100.0

In [74]:
train_label_hot.shape

(593089, 2)

In [75]:
train_dataset_reform.shape

(593089, 1)

In [76]:
num_steps = int(iteration_one_epoch*10)
loss_history = []

with tf.Session() as session:
# with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as session:
#with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_label_hot.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_reform[offset:(offset + batch_size), :]
        batch_labels = train_label_hot[offset:(offset + batch_size), :]

        # print(offset)
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions, added_summary, logits_check = session.run(
            [optimizer, loss, train_prediction, loss_summary, logits], feed_dict=feed_dict)

        # writer.add_summary(added_summary)
        loss_history.append(l)
        # print(offset, batch_size)
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print(batch_labels)
        print(predictions)
        # if (step % iteration_one_epoch == 0):
        # print("Minibatch loss at step %d: %f" % (step, l))
            # print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            # print(logits_check)
            # print(weights1.eval())
            # print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    
    # Do for the test batch
    acc = 0
    for step in range(int(iteration_one_epoch)):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_label_hot.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_reform[offset:(offset + batch_size), :]
        batch_labels = train_label_hot[offset:(offset + batch_size), :]

        # print(offset)
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        test_predictions = session.run(train_prediction, feed_dict=feed_dict)
        print(offset, offset + batch_size)
        print(batch_labels)
        print(test_predictions)
        batch_acc = accuracy(test_predictions, batch_labels)
        print("Minibatch accuracy: %.1f%%" % batch_acc)
        acc = acc + batch_acc

    print(acc)

Initialized
Minibatch loss at step 0: 4785019.500000
Minibatch accuracy: 50.0%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]
Minibatch loss at step 1: 0.000000
Minibatch accuracy: 100.0%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch loss at step 2: 0.000000
Minibatch accuracy: 100.0%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch loss at step 3: 0.000000
Minibatch accuracy: 100.0%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch loss at step 4: 0.000000
Minibatch accuracy: 100.0%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch loss at step 5: 0.000000
Minibatch accuracy: 100.0%
[[0. 1.]
 [

Minibatch loss at step 44: 168742.468750
Minibatch accuracy: 64.3%
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch loss at step 45: 296354.187500
Minibatch accuracy: 23.7%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]
Minibatch loss at step 46: 103039.625000
Minibatch accuracy: 76.7%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Minibatch loss at step 47: 125228.109375
Minibatch accuracy: 77.3%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Minibatch loss at step 48: 128355.445312
Minibatch accuracy: 75.6%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Minibatch loss at step 49: 63890.683594
Minibatch accuracy: 78.0%

Minibatch loss at step 88: 13402.914062
Minibatch accuracy: 74.8%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Minibatch loss at step 89: 74448.257812
Minibatch accuracy: 22.6%
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]
Minibatch loss at step 90: 28730.724609
Minibatch accuracy: 24.7%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]
Minibatch loss at step 91: 29361.978516
Minibatch accuracy: 78.1%
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Minibatch loss at step 92: 53222.906250
Minibatch accuracy: 78.4%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Minibatch loss at step 93: 52571.531250
Minibatch accuracy: 80.3%
[[1.

Minibatch loss at step 132: 62460.261719
Minibatch accuracy: 77.6%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Minibatch loss at step 133: 146253.968750
Minibatch accuracy: 66.2%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch loss at step 134: 81646.609375
Minibatch accuracy: 64.7%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch loss at step 135: 132275.281250
Minibatch accuracy: 75.8%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Minibatch loss at step 136: 208817.515625
Minibatch accuracy: 35.6%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
Minibatch loss at step 137: 195759.250000
Minibatch accuracy: 

Minibatch loss at step 176: 29064.439453
Minibatch accuracy: 64.6%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch loss at step 177: 24812.009766
Minibatch accuracy: 64.5%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch loss at step 178: 12237.407227
Minibatch accuracy: 74.8%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Minibatch loss at step 179: 48715.515625
Minibatch accuracy: 36.0%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
Minibatch loss at step 180: 15339.660156
Minibatch accuracy: 75.3%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Minibatch loss at step 181: 17798.101562
Minibatch accuracy: 63.7

...

Minibatch loss at step 888: 1063.992920
Minibatch accuracy: 55.1%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch loss at step 889: 539.702576
Minibatch accuracy: 49.7%
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch loss at step 890: 598.727783
Minibatch accuracy: 88.2%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Minibatch loss at step 891: 1198.189575
Minibatch accuracy: 73.0%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Minibatch loss at step 892: 6742.680176
Minibatch accuracy: 50.0%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Minibatch loss at step 893: 6189.633789
Minibatch accuracy: 50.0%
[[0. 1

Minibatch loss at step 933: 1940.360840
Minibatch accuracy: 64.6%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch loss at step 934: 966.877502
Minibatch accuracy: 67.0%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch loss at step 935: 1053.347900
Minibatch accuracy: 77.8%
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Minibatch loss at step 936: 1586.497192
Minibatch accuracy: 76.3%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Minibatch loss at step 937: 1861.954346
Minibatch accuracy: 76.7%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Minibatch loss at step 938: 1826.183350
Minibatch accuracy: 77.4%
[[1. 

Minibatch loss at step 978: 112.525688
Minibatch accuracy: 77.5%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Minibatch loss at step 979: 146.186371
Minibatch accuracy: 74.8%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Minibatch loss at step 980: 62.307514
Minibatch accuracy: 77.3%
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Minibatch loss at step 981: 301.918945
Minibatch accuracy: 65.1%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch loss at step 982: 291.638611
Minibatch accuracy: 62.7%
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch loss at step 983: 91.009552
Minibatch accuracy: 78.1%
[[0. 1.]
 [0

160110 166040
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 65.3%
166040 171970
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 65.4%
171970 177900
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 66.2%
177900 183830
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 64.8%
183830 189760
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 67.2%
189760 195690
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 64.5%
195690 201620
[[1. 1.]
 [1. 1.]
 [1. 1.]

468470 474400
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 64.8%
474400 480330
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 64.2%
480330 486260
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 66.1%
486260 492190
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 63.8%
492190 498120
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 65.9%
498120 504050
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 64.0%
504050 509980
[[1. 1.]
 [1. 1.]
 [1. 1.]

In [46]:
533772+59308

593080

In [62]:
train_label_hot.shape

(593089, 2)

In [46]:
predictions

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [52]:
batch_labels.shape

(5930, 2)

In [55]:
predictions[:3].shape

(3, 2)

In [63]:
(np.sum(predictions == batch_labels))/num_label/predictions.shape[0]

0.18591905564924116

Small NN

In [68]:
batch_size = int(num_samples/100)
hidden_size = 4
hidden_size2 = 4
hidden_size3 = 4
hidden_size4 = 4

# 20, 100, 500, 1000, 2000

# graph = tf.Graph()
# with tf.device('/device:GPU:0'):
# with graph.as_default():
# Input data. For the training data, we use a placeholder that will be fed
# at run time with a training minibatch.
tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, 1))
tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_label))
# tf_valid_dataset = tf.constant(valid_dataset)
tf_test_dataset = tf.placeholder(tf.float32, shape=(batch_size, num_label))
tf_test_labels = tf.placeholder(tf.float32, shape=(batch_size, num_label))

# Variables.
weights1 = tf.Variable(tf.truncated_normal([1, hidden_size]))
biases1 = tf.Variable(tf.zeros([hidden_size]))

weights2 = tf.Variable(tf.truncated_normal([hidden_size, num_label]))
biases2 = tf.Variable(tf.zeros([num_label]))

"""
weights2 = tf.Variable(tf.truncated_normal([hidden_size, hidden_size2]))
biases2 = tf.Variable(tf.zeros([hidden_size2]))

weights3 = tf.Variable(tf.truncated_normal([hidden_size2,  num_label]))
biases3 = tf.Variable(tf.zeros([ num_label]))


# weights4 = tf.Variable(tf.truncated_normal([hidden_size3, num_label]))
# biases4 = tf.Variable(tf.zeros([num_label]))
weights4 = tf.Variable(tf.truncated_normal([hidden_size3, hidden_size4]))
biases4 = tf.Variable(tf.zeros([hidden_size4]))

weights5 = tf.Variable(tf.truncated_normal([hidden_size4, num_label]))
biases5 = tf.Variable(tf.zeros([num_label]))
"""

# Training computation.
logits1 = tf.matmul(tf_train_dataset, weights1) + biases1  
relu_act_func = tf.nn.relu(logits1)
logits2 = tf.matmul(relu_act_func, weights2) + biases2

"""
relu_act_func2 = tf.nn.relu(logits2)
logits3 = tf.matmul(relu_act_func2, weights3) + biases3

relu_act_func3 = tf.nn.relu(logits3)
logits4 = tf.matmul(relu_act_func3, weights4) + biases4

relu_act_func4 = tf.nn.relu(logits4)
logits5 = tf.matmul(relu_act_func4, weights5) + biases5
"""

logits = logits2

# loss calculation
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits5))
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

# Predictions for the training, validation, and test data.
# train_prediction = tf.nn.softmax(logits5)
train_prediction = tf.round(tf.sigmoid(logits))

# loss = tf.reduce_mean(tf.losses.absolute_difference(labels=tf_train_labels, predictions=train_prediction))

loss_summary = tf.summary.scalar('loss', loss)

# Optimizer.
# optimizer = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)
optimizer = tf.train.AdamOptimizer(2).minimize(loss)

In [69]:
num_steps = int(iteration_one_epoch*10)
loss_history = []

with tf.Session() as session:
# with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as session:
#with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_label_hot.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_reform[offset:(offset + batch_size), :]
        batch_labels = train_label_hot[offset:(offset + batch_size), :]

        # print(offset)
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions, added_summary, logits_check = session.run(
            [optimizer, loss, train_prediction, loss_summary, logits], feed_dict=feed_dict)

        # writer.add_summary(added_summary)
        loss_history.append(l)
        # print(offset, batch_size)
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print(batch_labels)
        print(predictions)
        print(weights1.eval())
        # if (step % iteration_one_epoch == 0):
        # print("Minibatch loss at step %d: %f" % (step, l))
            # print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            # print(logits_check)
            # print(weights1.eval())
            # print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    
    # Do for the test batch
    acc = 0
    for step in range(int(iteration_one_epoch)):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_label_hot.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_reform[offset:(offset + batch_size), :]
        batch_labels = train_label_hot[offset:(offset + batch_size), :]

        # print(offset)
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        loss_test, test_predictions = session.run([loss, train_prediction], feed_dict=feed_dict)
        print(step, offset, offset + batch_size, loss_test)
        print(batch_labels)
        print(test_predictions)
        batch_acc = accuracy(test_predictions, batch_labels)
        print("Minibatch accuracy: %.1f%%" % batch_acc)
        acc = acc + batch_acc

    print(acc)
        

Initialized
Minibatch loss at step 0: 1720.878418
Minibatch accuracy: 50.0%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]
[[-0.96173817 -0.80049795  2.1028476  -1.4732914 ]]
Minibatch loss at step 1: 0.000000
Minibatch accuracy: 100.0%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[-0.96173817 -0.80049795  3.4429598  -2.8134031 ]]
Minibatch loss at step 2: 0.000000
Minibatch accuracy: 100.0%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[-0.96173817 -0.80049795  4.4788666  -3.84931   ]]
Minibatch loss at step 3: 0.000000
Minibatch accuracy: 100.0%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[-0.96173817 -0.80049795  5.3274217  -4.697865  ]]
Minibatch loss at step 4: 0.000000
Minibatch accur

Minibatch loss at step 37: 0.462261
Minibatch accuracy: 78.2%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -15.384248   -11.227346  ]]
Minibatch loss at step 38: 0.468001
Minibatch accuracy: 76.1%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -15.499875   -11.250741  ]]
Minibatch loss at step 39: 0.478378
Minibatch accuracy: 77.1%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -15.605117   -11.272034  ]]
Minibatch loss at step 40: 0.472071
Minibatch accuracy: 77.1%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -15.7008915  -11.291411  ]]
Minibatch loss at step 41: 0.498957
Minibatch ac

Minibatch loss at step 86: 0.497677
Minibatch accuracy: 75.1%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.643785   -11.4821825 ]]
Minibatch loss at step 87: 0.475460
Minibatch accuracy: 77.5%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.644775   -11.482383  ]]
Minibatch loss at step 88: 0.490573
Minibatch accuracy: 74.8%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.645672   -11.482564  ]]
Minibatch loss at step 89: 0.485454
Minibatch accuracy: 77.4%
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.646484   -11.482728  ]]
Minibatch loss at step 90: 0.497083
Minibatch ac

[[ -0.96173817  -0.80049795 -16.654108   -11.48427   ]]
Minibatch loss at step 134: 0.563116
Minibatch accuracy: 78.2%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.654118   -11.484272  ]]
Minibatch loss at step 135: 0.632938
Minibatch accuracy: 75.8%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.654125   -11.484274  ]]
Minibatch loss at step 136: 0.592046
Minibatch accuracy: 77.4%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.654133   -11.484276  ]]
Minibatch loss at step 137: 0.556539
Minibatch accuracy: 76.8%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.654139   -11

Minibatch loss at step 180: 0.484361
Minibatch accuracy: 75.3%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.654194   -11.484285  ]]
Minibatch loss at step 181: 0.489912
Minibatch accuracy: 76.6%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.654194   -11.484285  ]]
Minibatch loss at step 182: 0.479742
Minibatch accuracy: 76.3%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.654194   -11.484285  ]]
Minibatch loss at step 183: 0.495266
Minibatch accuracy: 75.5%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.654194   -11.484285  ]]
Minibatch loss at step 184: 0.476966
Minibat

...

[[ -0.96173817  -0.80049795 -16.654194   -11.484285  ]]
Minibatch loss at step 873: 0.496097
Minibatch accuracy: 75.3%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.654194   -11.484285  ]]
Minibatch loss at step 874: 0.473637
Minibatch accuracy: 77.5%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.654194   -11.484285  ]]
Minibatch loss at step 875: 0.488327
Minibatch accuracy: 74.8%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.654194   -11.484285  ]]
Minibatch loss at step 876: 0.486737
Minibatch accuracy: 77.0%
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.654194   -11

Minibatch loss at step 916: 0.513500
Minibatch accuracy: 78.0%
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.654194   -11.484285  ]]
Minibatch loss at step 917: 0.554090
Minibatch accuracy: 66.5%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[ -0.96173817  -0.80049795 -16.654194   -11.484285  ]]
Minibatch loss at step 918: 0.645995
Minibatch accuracy: 65.3%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[ -0.96173817  -0.80049795 -16.654194   -11.484285  ]]
Minibatch loss at step 919: 0.684218
Minibatch accuracy: 65.4%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[ -0.96173817  -0.80049795 -16.654194   -11.484285  ]]
Minibatch loss at step 920: 0.662568
Minibat

Minibatch accuracy: 77.1%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.654194   -11.484285  ]]
Minibatch loss at step 962: 0.478292
Minibatch accuracy: 76.2%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.654194   -11.484285  ]]
Minibatch loss at step 963: 0.490621
Minibatch accuracy: 76.0%
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.654194   -11.484285  ]]
Minibatch loss at step 964: 0.475883
Minibatch accuracy: 77.2%
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[ -0.96173817  -0.80049795 -16.654194   -11.484285  ]]
Minibatch loss at step 965: 0.494707
Minibatch accuracy: 74.9%
[[0. 1.]
 [0. 1.]


4 23720 29650 0.0039806594
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 100.0%
5 29650 35580 0.0039806594
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 100.0%
6 35580 41510 1.2076807
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 76.2%
7 41510 47440 1.5375363
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 70.1%
8 47440 53370 1.5679694
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 69.3%
9 53370 59300 1.6407044
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]

54 320220 326150 1.8299862
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 64.8%
55 326150 332080 1.7816178
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 66.1%
56 332080 338010 1.8489889
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 64.4%
57 338010 343940 1.7449518
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 66.5%
58 343940 349870 1.8742738
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Minibatch accuracy: 64.2%
59 349870 355800 1.7984915
[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.

In [246]:
#BATCH

In [ ]:
batch_size = int(num_samples)
hidden_size = 1024
hidden_size2 = 512
hidden_size3 = 16

# 20, 100, 500, 1000, 2000

# graph = tf.Graph()

#with graph.as_default():
# Input data. For the training data, we use a placeholder that will be fed
# at run time with a training minibatch.
tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, 1))
tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_label))
# tf_valid_dataset = tf.constant(valid_dataset)
# tf_test_dataset = tf.constant(test_dataset)

# Variables.
weights1 = tf.Variable(tf.truncated_normal([1, hidden_size]))
biases1 = tf.Variable(tf.zeros([hidden_size]))

weights2 = tf.Variable(tf.truncated_normal([hidden_size, hidden_size2]))
# biases2 = tf.Variable(tf.zeros([num_label]))
biases2 = tf.Variable(tf.zeros([hidden_size2]))

weights3 = tf.Variable(tf.truncated_normal([hidden_size2, hidden_size3]))
biases3 = tf.Variable(tf.zeros([hidden_size3]))

weights4 = tf.Variable(tf.truncated_normal([hidden_size3, num_label]))
biases4 = tf.Variable(tf.zeros([num_label]))

# Training computation.
logits1 = tf.matmul(tf_train_dataset, weights1) + biases1  
relu_act_func = tf.nn.relu(logits1)
logits2 = tf.matmul(relu_act_func, weights2) + biases2

relu_act_func2 = tf.nn.relu(logits2)
logits3 = tf.matmul(relu_act_func2, weights3) + biases3

relu_act_func3 = tf.nn.relu(logits3)
logits4 = tf.matmul(relu_act_func3, weights4) + biases4

loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits4))

# Predictions for the training, validation, and test data.
train_prediction = tf.nn.softmax(logits4)

# loss = tf.reduce_mean(tf.losses.absolute_difference(labels=tf_train_labels, predictions=train_prediction))

loss_summary = tf.summary.scalar('loss', loss)

# Optimizer.
optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
#optimizer = tf.train.AdamOptimizer(1).minimize(loss)

# Predictions for the training, validation, and test data.
# train_prediction = tf.nn.softmax(logits4)

#valid_prediction = tf.nn.softmax(
#    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
#test_prediction = tf.nn.softmax(
#    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

writer = tf.summary.FileWriter('./vehicle', tf.get_default_graph())

In [ ]:
# BATCH GRADIENT DESCENT
loss_history = []
num_steps = 2

with tf.Session() as session:
#with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_label_hot.shape[0] - batch_size)
    # Generate a minibatch.
    #batch_data = train_dataset_reform[offset:(offset + batch_size), :]
    #batch_labels = train_label_hot[offset:(offset + batch_size), :]
    batch_data = train_dataset_reform
    batch_labels = train_label_hot
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions, added_summary = session.run(
        [optimizer, loss, train_prediction, loss_summary], feed_dict=feed_dict)
    
    writer.add_summary(added_summary)
    loss_history.append(l)
    
    # if (step % iteration_one_epoch == 0):
    print("Minibatch loss at step %d: %f" % (step, l))
    print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
    # print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    # print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))